In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.applications import ResNet50
from IPython.display import clear_output
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import tensorflow as tf
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os

In [2]:
# ap = argparse.ArgumentParser()
# ap.add_argument("-d", "--dataset", required=True,
# 	help="path to input dataset")
# ap.add_argument("-m", "--model", required=True,
# 	help="path to output serialized model")
# ap.add_argument("-l", "--label-bin", required=True,
# 	help="path to output label binarizer")
# ap.add_argument("-e", "--epochs", type=int, default=25,
# 	help="# of epochs to train our network for")
# ap.add_argument("-p", "--plot", type=str, default="plot.png",
# 	help="path to output loss/accuracy plot")
# args = vars(ap.parse_args())

In [3]:
cd E:/CS/Graduation Project/-A-learning-based-semi-Humanoid-Robot-for-On-campus-Surveillance-main/Integrated

E:\CS\Graduation Project\-A-learning-based-semi-Humanoid-Robot-for-On-campus-Surveillance-main\Integrated


In [4]:
epochs=20
data_path="ResNet_Data"
out_path="Model"

In [5]:
# initialize the set of labels from the spots activity dataset we are
# going to train our network on
LABELS = set(['Fight','Normal','Playing-Cards','Smoking'])
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images("ResNet_Data"))

data = []
labels = []
# loop over the image paths
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]
	# if the label of the current image is not part of of the labels
	# are interested in, then ignore the image
	if label not in LABELS:
		continue
	# load the image, convert it to RGB channel ordering, and resize
	# it to be a fixed 224x224 pixels, ignoring aspect ratio
	clear_output(wait=True)
	print(imagePath)
	image = cv2.imread(imagePath)
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (200, 200))
	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)

ResNet_Data\Smoking\smoking_1127.jpg


In [6]:
# convert the data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, stratify=labels, random_state=42)

In [7]:
# loaded_model = tf.keras.models.load_model("ResNet.h5")

In [8]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")
# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [9]:
# load the ResNet-50 network, ensuring the head FC layer sets are left
# off
baseModel = ResNet50(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(200, 200, 3)))
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in baseModel.layers:
	layer.trainable = False

In [10]:
# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1)

mc = ModelCheckpoint('BestResNetmodel.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / epochs)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit(
	x=trainAug.flow(trainX, trainY, batch_size=32),
	steps_per_epoch=len(trainX) // 32,
	validation_data=valAug.flow(testX, testY),
	validation_steps=len(testX) // 32,
	epochs=20)

[INFO] compiling model...


E:\Anaconda\envs\notebook\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


[INFO] training head...
Epoch 1/20
388/388 [==============================] - 2021s 5s/step - loss: 0.8117 - accuracy: 0.6923 - val_loss: 0.3620 - val_accuracy: 0.8702
Epoch 2/20
388/388 [==============================] - 1956s 5s/step - loss: 0.4610 - accuracy: 0.8225 - val_loss: 0.2884 - val_accuracy: 0.8941
Epoch 3/20
388/388 [==============================] - 1930s 5s/step - loss: 0.3923 - accuracy: 0.8451 - val_loss: 0.2593 - val_accuracy: 0.9019
Epoch 4/20
388/388 [==============================] - 1953s 5s/step - loss: 0.3515 - accuracy: 0.8611 - val_loss: 0.2442 - val_accuracy: 0.9050
Epoch 5/20
388/388 [==============================] - 1938s 5s/step - loss: 0.3210 - accuracy: 0.8728 - val_loss: 0.2274 - val_accuracy: 0.9133
Epoch 6/20
388/388 [==============================] - 2061s 5s/step - loss: 0.3057 - accuracy: 0.8769 - val_loss: 0.2143 - val_accuracy: 0.9196
Epoch 7/20
388/388 [==============================] - 1966s 5s/step - loss: 0.2977 - accuracy: 0.8835 - val_loss

In [11]:
# serialize the model to disk
print("[INFO] serializing network...")
model.save("BigResNet.h5")
print("Saved...")
# serialize the label binarizer to disk
f = open("lb.pickle", "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] serializing network...


E:\Anaconda\envs\notebook\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Saved...


In [12]:
# model = tf.keras.models.load_model("E:/CS/Graduation Project/-A-learning-based-semi-Humanoid-Robot-for-On-campus-Surveillance-main/Integrated/ResNet.h5")
# model.layers[0].input_shape 

In [13]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX.astype("float32"), batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))
# plot the training loss and accuracy


[INFO] evaluating network...
               precision    recall  f1-score   support

        Fight       0.99      0.99      0.99      1020
       Normal       0.91      0.86      0.88      1060
Playing-Cards       0.99      0.99      0.99      1055
      Smoking       0.86      0.91      0.89      1013

     accuracy                           0.94      4148
    macro avg       0.94      0.94      0.94      4148
 weighted avg       0.94      0.94      0.94      4148



In [14]:
N = 10
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("")
plt.show()

ValueError: x and y must have same first dimension, but have shapes (10,) and (20,)